# Daly Weights, inflow and outflow of ICD-10 - dalys: Panel creation

This script runs a `sql` code to create a panel data, based on the `case_records_current` table, where we have each `ICD-10` member change of status, and the link between the status change with change in daly weights. 
The calculation involves a lot of cleaning,   such as desconsidering impetous changes of status or *cancelled* icds status as the first status input, etc, we leave that for the sql code, and here is just the script to 
run it and export to the redshift.

Steps:

    - run this notebook to upload to Redshift;
    
    - use the new panel to make calculations, new tables, dashes...
***

## 0. Setup

In [1]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [2]:
df = di.redshift.run_sql_query(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/queries/daly_weights_member_cid_panel_table.sql')
df.head(10)

,member_internal_code,case_id,seq_id,cid,cid_cap,cid_alice_cat,cid_status,cid_sev,added_at,daly_weight,delta_daly,daly_inflow,daly_outflow
0,NC1002P,18d366f7-7b01-4395-9551-a568d92ac700,1,J039,Capítulo X - Doenças do aparelho respiratório,Doenças bacterianas agudas,ACTIVE,DECOMPENSATED,2022-09-26 19:07:13.249774,0.000,NaN,0.000,0.0
1,NC1002P,f6248db0-c45b-4052-90ce-b505d2e1b500,1,E078,"Capítulo IV - Doenças endócrinas, nutricionais...",Transtornos da tireoide,ACTIVE,COMPENSATED,2022-05-10 15:10:15.285000,0.000,NaN,0.000,0.0
2,NC1006X,6beccc39-070f-47a6-80a9-c36cdf384c00,1,F410,Capítulo V - Transtornos mentais e comportamen...,Transtornos Ansiosos,ACTIVE,COMPENSATED,2021-11-09 12:19:19.628000,0.018,NaN,0.018,0.0
3,NC1006X,7f5ae321-2ad0-4f97-8a0e-1db273a80b00,1,F31,Capítulo V - Transtornos mentais e comportamen...,Transtorno Afetivo Bipolar,ACTIVE,COMPENSATED,2021-11-09 12:18:54.792000,0.032,NaN,0.032,0.0
4,NC1006X,a9026021-298a-4c7d-8e07-8ce8d04b5000,1,H521,Capítulo VII - Doenças do olho e anexos,Transtornos da refração e da acomodação,ACTIVE,COMPENSATED,2021-11-09 12:18:12.713000,0.003,NaN,0.003,0.0
5,NC1006X,b3bd4163-8219-4769-9575-0b4c9598cf00,1,L70,Capítulo XII - Doenças da pele e do tecido sub...,Acne,ACTIVE,COMPENSATED,2021-11-09 12:19:39.003000,0.000,NaN,0.000,0.0
6,NC1006X,fcc25483-6705-45cf-9801-a500ec3ce600,1,H522,Capítulo VII - Doenças do olho e anexos,Transtornos da refração e da acomodação,ACTIVE,COMPENSATED,2021-11-09 12:18:28.167000,0.003,NaN,0.003,0.0
7,NC100AL,73d6ef80-0aea-4d5a-b2a0-d5a1310f6000,1,Z720,Capítulo XXI - Fatores que influenciam o estad...,Fatores de risco,ACTIVE,DECOMPENSATED,2022-07-29 16:12:00.816000,0.000,-0.025,0.000,0.0
8,NC100AL,73d6ef80-0aea-4d5a-b2a0-d5a1310f6000,1,Z720,Capítulo XXI - Fatores que influenciam o estad...,Fatores de risco,ACTIVE,DECOMPENSATED,2022-07-29 16:12:00.816000,0.025,NaN,0.025,0.0
9,NC100AL,a3de6582-d57c-40fe-b8db-946cce4b5000,1,E55,"Capítulo IV - Doenças endócrinas, nutricionais...",Nutricional,ACTIVE,DECOMPENSATED,2022-11-15 09:52:58.178398,0.002,NaN,0.002,0.0


## 3. Export

In [3]:
di.redshift.insert_table(df, table_name='daly_weights_member_cid_panel_table', if_exists='replace')

Table daly_weights_member_cid_panel_table inserted successfully, with 58833 rows.
